In [1]:
# Import Dependencies
import pandas as pd 
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
from flask import Flask, jsonify, render_template

# Create the connection engine
engine = create_engine("sqlite:///DataSets/belly_button_biodiversity.sqlite")
conn = engine.connect()

# Use SQLAlchemy `automap_base()` to reflect your tables into classes
Base = automap_base()
Base.prepare(engine, reflect=True)

# Save a reference to those classes called `Station` and `Measurement`
otu = Base.classes.otu
samples = Base.classes.samples
samples_metadata = Base.classes.samples_metadata

# Create a session for the engine to manipulate data
session = Session(engine)

In [17]:
name_query = session.query(samples_metadata.SAMPLEID).all()
names_list = list(np.ravel(name_query))
names = []
for name in names_list:
    names.append(f"BB_{name}")
names

['BB_940',
 'BB_941',
 'BB_943',
 'BB_944',
 'BB_945',
 'BB_946',
 'BB_947',
 'BB_948',
 'BB_949',
 'BB_950',
 'BB_952',
 'BB_953',
 'BB_954',
 'BB_955',
 'BB_956',
 'BB_958',
 'BB_959',
 'BB_960',
 'BB_961',
 'BB_962',
 'BB_963',
 'BB_964',
 'BB_966',
 'BB_967',
 'BB_968',
 'BB_969',
 'BB_970',
 'BB_971',
 'BB_972',
 'BB_973',
 'BB_974',
 'BB_975',
 'BB_978',
 'BB_1233',
 'BB_1234',
 'BB_1235',
 'BB_1236',
 'BB_1237',
 'BB_1238',
 'BB_1242',
 'BB_1243',
 'BB_1246',
 'BB_1253',
 'BB_1254',
 'BB_1258',
 'BB_1259',
 'BB_1260',
 'BB_1264',
 'BB_1265',
 'BB_1273',
 'BB_1275',
 'BB_1276',
 'BB_1277',
 'BB_1278',
 'BB_1279',
 'BB_1280',
 'BB_1281',
 'BB_1282',
 'BB_1283',
 'BB_1284',
 'BB_1285',
 'BB_1286',
 'BB_1287',
 'BB_1288',
 'BB_1289',
 'BB_1290',
 'BB_1291',
 'BB_1292',
 'BB_1293',
 'BB_1294',
 'BB_1295',
 'BB_1296',
 'BB_1297',
 'BB_1298',
 'BB_1308',
 'BB_1309',
 'BB_1310',
 'BB_1374',
 'BB_1415',
 'BB_1439',
 'BB_1441',
 'BB_1443',
 'BB_1486',
 'BB_1487',
 'BB_1489',
 'BB_1490',
 

In [20]:
description_query = session.query(otu.lowest_taxonomic_unit_found).all()
description_list = list(np.ravel(description_query))
description_list

['Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus',
 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus',
 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum',
 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter',
 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter',
 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bacteria',
 'Bact

In [63]:
def get_metaData(sample):
    metadata_query = session.query(samples_metadata.AGE, samples_metadata.BBTYPE, samples_metadata.ETHNICITY,\
                                   samples_metadata.GENDER, samples_metadata.LOCATION, samples_metadata.SAMPLEID)\
                                    .filter(samples_metadata.SAMPLEID == sample).first()
    metadata_info = list(np.ravel(metadata_query))
    metadata_dict = {}
    metadata_dict["AGE"] = metadata_info[0]
    metadata_dict["BBTYPE"] = metadata_info[1]
    metadata_dict["ETHNICITY"] = metadata_info[2]
    metadata_dict["GENDER"] = metadata_info[3]
    metadata_dict["LOCATION"] = metadata_info[4]
    metadata_dict["SAMPLIE"] = metadata_info[5]
    
    return metadata_dict


                               

In [64]:
get_metaData("940")

{'AGE': '24',
 'BBTYPE': 'I',
 'ETHNICITY': 'Caucasian',
 'GENDER': 'F',
 'LOCATION': 'Beaufort/NC',
 'SAMPLIE': '940'}

In [80]:
def weeklyWashingFreq(sample):
    search_sample = sample[3:]
    wash_query = session.query(samples_metadata.WFREQ).filter(samples_metadata.SAMPLEID == search_sample).first()
    wfreq = list(np.ravel(wash_query))
    return wfreq[0]

In [81]:
weeklyWashingFreq("BB_941")

1

In [118]:
def otu_values(sample):
    otu_query = session.query(samples.otu_id, getattr(samples, sample)).order_by(getattr(samples, sample).desc()).all()
    otuValues = []
    id_list = []
    values_list = []
    otuValues_dict = {}
    for x in otu_query:
        (otu_ids, sample_values) = x
        id_list.append(otu_ids)
        values_list.append(sample_values)
    otuValues_dict["otu_ids"] = id_list
    otuValues_dict["sample_values"] = values_list
    otuValues.append(otuValues_dict)
    return otuValues

In [119]:
otu_values("BB_940")

[{'otu_ids': [1167,
   2859,
   482,
   2264,
   41,
   1189,
   352,
   189,
   1977,
   2318,
   3450,
   874,
   1959,
   2191,
   1950,
   2077,
   2275,
   944,
   2184,
   2244,
   2024,
   2419,
   2811,
   165,
   2782,
   2011,
   2247,
   2396,
   830,
   1795,
   2964,
   2722,
   307,
   2178,
   2908,
   1193,
   1208,
   2039,
   2167,
   1274,
   2737,
   2739,
   833,
   907,
   1314,
   1962,
   2186,
   2335,
   2936,
   121,
   159,
   170,
   258,
   259,
   340,
   342,
   357,
   412,
   513,
   725,
   1169,
   1232,
   1497,
   1498,
   1503,
   1505,
   1960,
   1968,
   2065,
   2110,
   2188,
   2235,
   2291,
   2342,
   2350,
   2475,
   2483,
   2491,
   2546,
   2571,
   11,
   19,
   22,
   24,
   39,
   42,
   79,
   81,
   84,
   93,
   102,
   122,
   133,
   154,
   185,
   198,
   226,
   235,
   244,
   246,
   300,
   314,
   320,
   327,
   328,
   359,
   361,
   373,
   381,
   386,
   400,
   404,
   407,
   414,
   564,
   593,
   620,
   650